<a id="intro"></a>
# Исследование для кредитного скоринга

**Описание проекта**

Заказчик — кредитный отдел банка. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цели исследования** - ответить на вопросы:
1. [Есть ли зависимость между количеством детей и возвратом кредита в срок?](#children)
2. [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#family_status)
3. [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#total_income_category)
4. [Как разные цели кредита влияют на его возврат в срок?](#purpose_category)
5. [Каковы возможные причины появления пропусков в исходных данных?](#data_gaps)
6. [Каким значением заполнять пропуски  в количественных переменных?](#median_value)

 👉 **[Итоги исследования здесь](#research_results)**

**Описание данных**
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Общая информация о данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в data
try:
    data = pd.read_csv('https://github.com/AlexSlobodskoj/AlexSlobodskoj/blob/f04ed75e7864185cfc734f8c6c16945b246d8abe/Credit_Bank/data.csv')
except:
    data = pd.read_csv('/Users/alexslobodskoj/data.csv')

In [2]:
# вывод первых 10 строчек датафрейма data на экран
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# вывод основной информации о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Вывод количество пропущенных значений для каждого столбца.

In [4]:
# используется комбинация двух методов
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем в [п.2.3](#dropna)

Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. 

Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех значений `total_income` с тем же типом занятости.


In [5]:
# заполнение пропусков в столбце total_income медианными значениями по типу дохода
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Такой артефакт - отрицательное количество дней трудового стажа в столбце `days_employed`. 

**Заменим все отрицательные значения трудового стажа положительными с помощью метода `abs()`**

In [6]:
# замена отрицательных значений в столбце days_employed
data['days_employed'] = data['days_employed'].abs()

# проверим информацию в столбце days_employed
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed`.**

In [7]:
# медианное значение трудового стажа days_employed для каждого типа занятости income_type
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. В рамках этого исследования не будем их исправлять.

**Оценим перечень уникальных значений количества детей в семье.**

In [8]:
# вывод уникальных значений в столбце children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Оценим количество строк, в которых встречаются такие аномальные значения

In [9]:
# присвоим соответствующим переменным информацию о датафреймах: исходном и отфильтрованном
original_rows = data.shape[0]
filtered_rows = data[(data['children'] == -1) | (data['children'] == 20)].shape[0]

# посчитаем долю отфильтрованных строк и выведем на экран
proportion = filtered_rows / original_rows
print(f'Количество аномальных строк: {filtered_rows}')
print(f'Доля аномальных строк: {proportion:.2%}')

Количество аномальных строк: 123
Доля аномальных строк: 0.57%


Учитывая незначительную долю аномальных строк - удалим их из исходного датафрейма

In [10]:
# удаление строк в которых в колонке children значения -1 или 20
data.drop(data[(data['children'] == -1) | (data['children'] == 20)].index, inplace=True)

# обновление индексов
data.reset_index(drop=True, inplace=True)

# проверка удаления артефактов в столбце children
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

<a id="dropna"></a>
### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
# заполнение пропусков в столбце days_employed медианными значениями по типу дохода
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски в столбцах`days_employed` и `total_income`заполнены.

In [12]:
# проверка пропущенных значений для total_income и days_employed
data[['days_employed','total_income']].isna().sum()

days_employed    0
total_income     0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [13]:
# замена типа даннах в total_income на int
data['total_income'] = data['total_income'].astype(int)

# проверка замены
data['total_income'].dtype

dtype('int64')

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце** `education`.

Оценим перечень уровней образования заемщиков

In [14]:
# вывод уникальных значений столбца education с количеством
data['education'].value_counts()

education
среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: count, dtype: int64

В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [15]:
# приведение данных в столбце education к нижнему регистру
data['education'] = data['education'].str.lower()

# проверка нового перечня уровней образования заемщиков
data['education'].value_counts()

education
среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: count, dtype: int64


**Удаление строк-дубликатов в данных**

In [16]:
# количество строк-дубликатов
data.duplicated().sum()

71

In [17]:
# удаление дубликатов
data = data.drop_duplicates()

# проверка отсутствия дубликатов
data.duplicated().sum()

0

### Категоризация данных

**Категоризация ежемесячного дохода заемщиков** 

На основании диапазонов, указанных банком, создадим в датафрейме столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`...

In [18]:
# функция для возврата наименования категории уровня дохода
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
# заполнение нового столбца total_income_category данными о категории уровня дохода
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Категоризация целей получения кредита.**

In [20]:
# вывод уникальных целей получения кредита из столбца `purpose`
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`.

In [21]:
# функция для создания нового списка категорий
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'
        
# заполнение нового столбца категорий целей получения кредита
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуем данные и ответим на вопросы

<a id="children"></a>
### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на вопрос проверим зависимость наличия задолженности по возврату кредитов `debt` от количества детей `children`

**Шаг 1. Обзор данных о наличии задолженности по кредиту**

In [22]:
# проверка уникальных значений в столбце debt
data['debt'].unique()

array([0, 1])

В столбце `debt` два целочисленных значения 0 или 1.

Очевидно:

- 1 - была задолженность
- 0 - задолженности не было

**Шаг 2. Сгруппируем данные по количеству детей и вычислим по каждой группе:**
* количество выданных кредитов 
* количество задолженностей по возврату кредита
* процент наличия задолженности по кредиту

In [23]:
# вывод таблицы для принятия решений
# count - количества кредитов, 
# sum - количество задолженностей, 
# rate_debt - процент задолженности
try:
    display(data.groupby('children').agg({
        'debt': ['count', 'sum', ('rate_debt', lambda x: f'{x.sum() / x.count():.2%}')]
    }))
except:
    pass
# для вычисления rate_debt подходит и метод mean (среднее значение)
# частное x.sum() / x.count() используется для акцента на доле просроченных кредитов от всех выданных

debt                
          count   sum rate_debt
children                       
0         14091  1063     7.54%
1          4808   444     9.23%
2          2052   194     9.45%
3           330    27     8.18%
4            41     4     9.76%
5             9     0     0.00%

Определим репрезентативность каждой группы по количеству детей

In [24]:
# # вывод таблицы доля каждой группы по количеству детей
# count - количества кредитов, 
# rate_count - доля от общего количества кредитов
try:
    display(data.groupby('children').agg({
        'debt': ['count', ('rate_count', lambda x: f'{x.count() / len(data['debt']):.2%}')]
    }))
except:
    pass

debt           
          count rate_count
children                  
0         14091     66.06%
1          4808     22.54%
2          2052      9.62%
3           330      1.55%
4            41      0.19%
5             9      0.04%

<div style="border:solid gray 1px; padding: 20px">

**Выводы:** 

1. Существует определенная зависимость между количеством детей у заемщика и возвратом кредита в срок.
    
    
2. Заемщики без детей имеют процент задолженности по возврату кредита **7.54%**. Это может указывать на то, что у людей без детей меньше непредвиденных расходов и они надежны при возврате кредитов.
    
    
3. Заемщики с одним ребенком имеют процент задолженности **9.23%**, что выше, чем у заемщиков без детей. Это может свидетельствовать о том, что наличие одного ребенка может повлиять на способность заемщика вовремя возвращать кредиты.
    
    
4. Заемщики с двумя детьми имеют процент задолженности **9.45%**, что также выше, чем у заемщиков без детей. Это может указывать на то, что у заемщиков с двумя детьми могут быть дополнительные финансовые обязательства, что влияет на возврат кредитов.
    
    
5. Заемщики с большим количестом детей:
- с тремя детьми имеют процент задолженности **8.18%**,
- с четырьмя детьми имеют самый высокий процент задолженности **9.76%**
- с пятью детьми не имеют задолженности
- данных по этим категориям недостаточно для анализа. Чтобы многодетная категория стала репрезентативной её доля должна быть более 2%. 

<a id="family_status"></a>
### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос проверим зависимость наличия задолженности по возврату кредита `debt` от семейного положения `family_status`

**Шаг 1. Обзор данных о семейном положении**

In [25]:
# проверка уникальные значения в столбце family_status
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Для удобства приведем наименование всех категорий семейного положения в нижний регистр

In [26]:
# приведение значений столбца family_status в нижний регистр 
data['family_status'] = data['family_status'].str.lower()

**Шаг 2. Сгруппируем данные по семейному положению и вычислим по каждой группе:**
* количество выданных кредитов 
* количество задолженностей по возврату кредита
* процент наличия задолженности по кредиту.

Метод `pivot_table` используется для демонстрации использования и в таких случаях.

In [27]:
#вывод сводной таблицы для принятия решений
# count - количества кредитов, 
# sum - количество задолженностей, 
# rate_debt - процент задолженности
try:
    pivot = data.pivot_table(index='family_status', values='debt', 
                         aggfunc={'debt': ['count', 'sum', ('rate_debt',lambda x: f'{x.sum() / x.count():.2%}')]},
                         fill_value=0)
except:
    pass

# изменение порядка столбцов
pivot = pivot.reindex(columns=['count', 'sum', 'rate_debt'])

# сортировка  по rate_debt для удобства анализа
pivot.sort_values(by='rate_debt', ascending=True)

,count,sum,rate_debt
family_status,,,
вдовец / вдова,951,63,6.62%
в разводе,1189,84,7.06%
женат / замужем,12261,927,7.56%
гражданский брак,4134,385,9.31%
не женат / не замужем,2796,273,9.76%


<div style="border:solid gray 1px; padding: 20px">

**Выводы:** 

1. Существует зависимость между семейным положением заёмщика и возвратом кредита в срок. 
    
    
2. Заемщики, являющиеся вдовцами или вдовами, имеют самый низкий процент задолженности по возврату кредита **6.62%**. Это может указывать на то, что люди, потерявшие супруга/супругу, более ответственны и надежны при возврате кредитов.
    
    
3. Заемщики, находящиеся в разводе, имеют процент задолженности **7.06%**. Это ниже, чем у заемщиков в гражданском браке или не состоящих в браке. Это может свидетельствовать о том, что люди, пережившие развод, могут быть более организованными и ответственными в финансовых вопросах.
    
    
4. Заемщики, находящиеся в браке, имеют процент задолженности **7.56%**, что также ниже, чем у заемщиков в гражданском браке или не состоящих в браке. Это может указывать на то, что семейное положение может влиять на финансовую ответственность заемщика.
    
    
5. Заемщики, находящиеся в гражданском браке, имеют процент задолженности **9.31%**, что выше, чем у заемщиков в предыдущих категориях. Это может указывать на то, что люди в гражданском браке могут иметь больше финансовых трудностей или менее стабильные отношения, что отражается на возврате кредитов.
    
    
5. Заемщики, не состоящие в браке, имеют самый высокий процент задолженности **9.76%**. Это может указывать на то, что люди, не имеющие официального семейного статуса, могут иметь менее стабильные финансовые обязательства или меньше мотивации для возврата кредитов.

<a id="total_income_category"></a>
### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

#### Анализ категорий уровня дохода, предложенных банком

Для ответа на вопрос проверим зависимость наличия задолженности по возврату кредитов `debt` от предложенных кредитным отделом и рассчитанных ранее категорий уровня дохода в столбце `total_income_category`

**Сгруппируем данные по категориям уровня дохода и вычислим по каждой группе:**
* количество выданных кредитов 
* количество задолженностей по возврату кредита
* процент наличия задолженности по кредиту.

In [28]:
# вывод таблицы для принятия решений
# count - количества кредитов, 
# sum - количество задолженностей, 
# rate_debt - процент задолженности
try:
    display(data.groupby('total_income_category').agg({
        'debt': ['count', 'sum', ('rate_debt', lambda x: f'{x.sum() / x.count():.2%}')]
    }).sort_index(ascending=False).rename(index={
    'A': 'более 1 млн - A',
    'B': 'от 200 тыс до 1 млн - B',
    'C': 'от 50 тыс до 200 тыс - C',
    'D': 'от 30 тыс до 50 тыс - D',
    'E': 'до 30 тыс - E'
    }))
except:
    pass

debt                
                          count   sum rate_debt
total_income_category                          
до 30 тыс - E                22     2     9.09%
от 30 тыс до 50 тыс - D     349    21     6.02%
от 50 тыс до 200 тыс - C  15921  1353     8.50%
от 200 тыс до 1 млн - B    5014   354     7.06%
более 1 млн - A              25     2     8.00%

<div style="border:solid gray 1px; padding: 20px">

**Выводы:**

1. Cуществует определенная зависимость между уровнем дохода и возвратом кредита в срок.
    
    
2. Заемщики с доходом **менее 30 тыс** имеют самый высокий процент задолженности по возврату кредита **9.09%**. Это может указывать на то, что люди с низким уровнем дохода чаще имеют проблемы с возвратом кредитов. Учитывая малое количество таких заемщиков рекомендуется провести дополнительное исследование.
    

3. Заемщики с доходом **от 30 тыс до 50 тыс** имеют самый низкий процент задолженности **6.02%**, что может указывать на то, что люди с доходом в этом диапазоне более ответственны при возврате кредитов.
    
        
4. Заемщики с доходом **от 50 тыс до 200 тыс** также имеют высокий процент задолженности **8.50%**. Это может свидетельствовать о том, что даже средний уровень дохода не всегда гарантирует своевременный возврат кредита. И таких заемщиков нужно оценивать по другим категориям: наличие детей, семейное положение ...
    
    
5. Заемщики с доходом **от 200 тыс до 1 млн** имеют низкий процент задолженности **7.06%**. Это может говорить о том, что люди с высоким уровнем дохода более надежны при возврате кредитов.
    
    
6. Заемщики с доходом **более 1 млн** имеют процент задолженности **8%**, что выше чем у предыдущей категории. Но малое количество таких заемщиков возможно искажает результаты исследования.

#### Анализ альтернативных категорий уровня дохода

Предложим диапазоны категорий уровня дохода, более равномерно разделяющие заемщиков и дающие реперезентативную выборку по каждой из категорий:

- 0–120000 — `'E'`;
- 120001–150000 — `'D'`;
- 150001–200000 — `'C'`;
- 200001–250000 — `'B'`;
- 250001 и выше — `'A'`.

Новые диапазоны запишем в столбец `total_income_range`

In [29]:
# для создания новых диапазонов изпользуем метод cut() библиотеки pandas
cut_labels = ['E', 'D', 'C', 'B', 'A']
cut_bins = [0, 120000, 150000, 200000, 250000, data['total_income'].max()]
data['total_income_range'] = pd.cut(data['total_income'], bins=cut_bins, labels=cut_labels)

# вывод таблицы на экран
try:
    display(data.groupby('total_income_range', observed=True).agg({
        'debt': ['count', 'sum', ('rate_debt', lambda x: f'{x.sum() / x.count():.2%}')]
    }).rename(index={
    'A': 'более 250 тыс - A',
    'B': 'от 200 тыс до 250 тыс - B',
    'C': 'от 150 тыс до 200 тыс - C',
    'D': 'от 120 тыс до 150 тыс - D',
    'E': 'до 120 тыс - E'
    }))
except:
    pass

debt               
                          count  sum rate_debt
total_income_range                            
до 120 тыс - E             7196  585     8.13%
от 120 тыс до 150 тыс - D  4358  388     8.90%
от 150 тыс до 200 тыс - C  4738  403     8.51%
от 200 тыс до 250 тыс - B  2242  162     7.23%
более 250 тыс - A          2797  194     6.94%

<div style="border:solid gray 1px; padding: 20px">

**Выводы:**

1. Информация о задолженности по возврату кредита по новым категориям уровня дохода более равномерно включает в себя заемщиков и уменьшает искажение результатов исследования.

2. С увеличением уровня дохода от **150 тыс** уменьшается процент задолженности по возврату кредита. Это может говорить о том, что люди с высоким уровнем дохода более надежны при возврате кредитов.

<a id="purpose_category"></a>
### Как разные цели кредита влияют на его возврат в срок?

Для ответа на вопрос проверим зависимость наличия задолженности по возврату кредитов `debt` от рассчитанных ранее категорий целей кредита в столбце `purpose_category`

**Сгруппируем данные по категориям целей кредита и вычислим по каждой группе:**
* количество выданных кредитов 
* количество задолженностей по возврату кредита
* процент наличия задолженности по кредиту.

In [30]:
#вывод таблицы для принятия решений
# count - количества кредитов, 
# sum - количество задолженностей, 
# rate_debt - процент задолженности
try:
    result = data.groupby('purpose_category').agg({
        'debt': ['count', 'sum', ('rate_debt', lambda x: f'{x.sum() / x.count():.2%}')]
    })
except:
    pass

# сортировка  по rate_debt для удобства анализа
result.sort_values(by=('debt', 'rate_debt'), ascending=True)

debt               
                          count  sum rate_debt
purpose_category                              
операции с недвижимостью  10751  780     7.26%
проведение свадьбы         2313  183     7.91%
получение образования      3988  369     9.25%
операции с автомобилем     4279  400     9.35%

<div style="border:solid gray 1px; padding: 20px">

**Выводы:** 

1. **"Операции с недвижимостью"** имеют самый низкий процент задолженности по возврату кредита **7.26%**. Это может указывать на то, что заемщики, берущие кредит на операции с недвижимостью, более ответственны и надежны при возврате кредитов.
    
    
2. **"Проведение свадьбы"** имеет процент задолженности **7.91%**, что немного выше, чем у заемщиков на операции с недвижимостью. Это может свидетельствовать о том, что заемщики, берущие кредит на свадьбу, могут иметь некоторые финансовые трудности, но в целом возвращают кредиты в срок.
    
    
3. **"Получение образования"** имеет процент задолженности **9.25%**, что выше, чем у заемщиков на операции с недвижимостью и свадьбу. Это может указывать на то, что заемщики, берущие кредит на образование, могут сталкиваться с финансовыми трудностями из-за высоких дополнительных расходов на обучение.
    
    
4. **"Операции с автомобилем"** имеют самый высокий процент задолженности **9.35%**. Это может указывать на то, что заемщики, берущие кредит на автомобиль, чаще имеют проблемы с возвратом кредитов, возможно из-за дополнительных расходов на обслуживание и содержание автомобиля.

<a id="data_gaps"></a>
### Возможные причины появления пропусков в исходных данных.

Одинакове количество пропусков в столбцах **«общий трудовой стаж в днях»** и **«ежемесячный доход»** может указывать на:
- **Особенность процесса сбора данных**. Например, использовался определённый метод сбора данных, который не смог получить информацию о трудовом стаже и доходе одновременно для некоторых клиентов.
- **Взаимозависимость данных**. Например, отсутствие информации о трудовом стаже исключает автоматическую проверку наличия информации о ежемесячном доходе.

Возможные причины:

1. **Новые клиенты**: Возможно, некоторые клиенты являются новыми и не предоставили достаточной информации о своем трудовом стаже и доходе на момент заполнения данных. Новые клиенты могут быть менее склонны к предоставлению подробной информации о своей трудовой и финансовой истории.


2. **Специфика данных**: Иногда банки могут не иметь доступа к полной информации о трудовом стаже и доходе клиента, особенно если клиент работает неофициально или не предоставил необходимые документы.


3. **Ошибки в сборе данных**: Возможно, при сборе данных произошли ошибки, которые привели к пропускам в столбцах "общий трудовой стаж в днях" и "ежемесячный доход". Это может быть вызвано недостаточной внимательностью при заполнении данных или техническими проблемами при вводе данных.


4. **Нежелание предоставлять информацию**: Некоторые клиенты могут не желать предоставлять информацию о своем трудовом стаже и доходе по различным причинам, таким как конфиденциальность данных, недоверие к финансовым учреждениям или просто личные предпочтения.

<a id="median_value"></a>
### Заполнять пропуски медианным значением — лучшее решение для количественных переменных.

Заполнение пропусков медианным значением является лучшим решением для количественных переменных по нескольким причинам:


1. **Сохранение распределения данных**: Медиана - центральная точка набора данных и меньше подвержена влиянию выбросов, чем среднее значение. Это позволяет сохранить исходное распределение данных и избежать искажения результатов из-за экстремальных значений.


2. **Устойчивость к выбросам**: Медианное значение менее чувствительно к экстремальным значениям, чем среднее, что делает его более устойчивым к ошибкам и выбросам в данных. Это особенно важно, когда данные содержат ошибки или аномалии.


3. **Сохранение структуры данных**: Медиана лучше отражает центральную тенденцию данных, сохраняя при этом структуру распределения. Это важно для сохранения внутренней логики и взаимосвязей между переменными.


4. **Простота интерпретации**: Медианное значение легко интерпретировать, оно даёт представление о типичном значении переменной без влияния экстремальных значений.


5. **Применимость к широкому спектру данных**: Метод подходит для широкого спектра количественных данных, включая те, где распределение сильно искажено или присутствуют выбросы.

<a id="research_results"></a>
## Итоги сследования

Результаты исследования могут быть использованы для построения модели кредитного скоринга, оценивающего способность потенциального заёмщика вернуть кредит банку.

**Установлено**:


1. Существует зависимость между количеством детей у заемщика и возвратом кредита в срок, причем чем больше детей у заемщика, тем выше вероятность задолженности по возврату кредита. Однако, разница в процентах задолженности между группами заёмщиков с разным количеством детей невелика, и для более точного определения зависимости требуется дополнительный анализ. 


2. Существует зависимость между семейным положением заемщика и возвратом кредита в срок, причем люди, находившиеся/находящиеся в стабильных отношениях (вдовцы/вдовы, разведенные, женатые/замужние), имеют более низкий процент задолженности по возврату кредита.


3. Существует зависимость между уровнем дохода и возвратом кредита в срок, причем люди с более высоким уровнем дохода имеют более низкий процент задолженности по возврату кредита. Это может указывать на то, что заёмщики с более высоким уровнем дохода более ответственно подходят к возврату кредитов.


4. Цель кредита может влиять на его возврат в срок. Заемщики, берущие кредит на операции с недвижимостью, обычно имеют более низкий процент задолженности, в то время как заемщики, берущие кредит на автомобиль или образование, чаще имеют проблемы с возвратом кредитов.

**Рекомендации**:

1. Для более точной интерпретации зависимостей проанализировать и скорректировать информацию о заемщиках, у которых "количество детей в семье" равно `-1` и `20` 


2. По-возможности обогатить информацию о заемщиках, у которых не заполнена информация о трудовом стаже и ежемесячном доходе.


3. Провести дополнительные исследования о зависимости возврата кредита в срок от уровня образования и возраста заемщика.

[Вернуться в начало](#intro)